In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import io
import string
import re
from sklearn.model_selection import train_test_split

In [5]:
en_filename = "dataset/train.en.txt"
vi_filename = "dataset/train.vi.txt"

raw_en_lines = open(en_filename, encoding='utf-8').read().strip().split("\n")
raw_vi_lines = open(vi_filename, encoding='utf-8').read().strip().split("\n")

In [6]:
exclude = list(string.punctuation) + list(string.digits)

def preprocess(sentence):
    sent = sentence.lower()
    sent = sent.strip()
    sent = re.sub("'", " ", sent)
    sent = re.sub("\s+", " ", sent)
    sent = ''.join([char for char in sent if char not in exclude])
    sent = "<start> " + sent + " <end>"
    return sent

In [7]:
en_lines = []
vi_lines = []
min_len, max_len = 10, 14

for eline, vline in zip(raw_en_lines, raw_vi_lines):
    eline = preprocess(eline)
    vline = preprocess(vline)
    if min_len < len(eline.split(" ")) < max_len and min_len < len(vline.split(" ")) < max_len:
        en_lines.append(eline)
        vi_lines.append(vline)
        
print(len(en_lines))

6016


In [29]:
class Language():
    def __init__(self, lines):
        self.lines = lines
        self.word2id = {}
        self.id2word = {}
        self.vocab = set()
        self.max_len = 0
        self.min_len = 0
        self.vocab_size = 0
        self.init_language_params()

    def init_language_params(self):
        for line in self.lines:
            self.vocab.update(line.split(" "))
        self.word2id['<pad>'] = 0
        for id, word in enumerate(self.vocab):
            self.word2id[word] = id + 1
        for word, id in self.word2id.items():
            self.id2word[id] = word
        self.max_len = max([len(line.split(" ")) for line in self.lines])
        self.min_len = min([len(line.split(" ")) for line in self.lines])
        self.vocab_size = len(self.vocab) + 1
            
    def sentence_to_vector(self, sent):
        return np.array([self.word2id[word] for word in sent.split(" ")])
            
    def vector_to_sentence(self, vector):
        return " ".join([self.id2word[id] for id in vector])

In [9]:
inp_lang = Language(en_lines)
tar_lang = Language(vi_lines)
print(inp_lang.max_len, inp_lang.min_len)
print(tar_lang.max_len, tar_lang.min_len)

inp_vector = [inp_lang.sentence_to_vector(line) for line in inp_lang.lines]
tar_vector = [tar_lang.sentence_to_vector(line) for line in tar_lang.lines]

inp_tensor = tf.keras.preprocessing.sequence.pad_sequences(inp_vector, inp_lang.max_len, padding='post')
tar_tensor = tf.keras.preprocessing.sequence.pad_sequences(tar_vector, tar_lang.max_len, padding='post')
print(inp_tensor.shape, tar_tensor.shape)

13 11
13 11
(6016, 13) (6016, 13)


In [10]:
i = 3
print(inp_tensor[i], type(inp_tensor[i]))
sent = inp_lang.vector_to_sentence(inp_tensor[i]) 
print(sent, type(sent))
vector = inp_lang.sentence_to_vector(inp_lang.vector_to_sentence(inp_tensor[i]))
print(vector, type(vector))
for inp, tar in zip(inp_tensor, tar_tensor):
    print(inp_lang.vector_to_sentence(inp))
    print(tar_lang.vector_to_sentence(tar))
    

Streaming output truncated to the last 5000 lines.
<start> audience      and the month  <end> <pad>
<start> khán giả      tháng  <end> <pad> <pad>
<start> good  how about the person behind her  <end> <pad> <pad>
<start> tốt  còn người ngồi phía sau cô  <end> <pad> <pad>
<start> audience      and the month  <end> <pad>
<start> khán giả      tháng  <end> <pad> <pad>
<start> so  without any more stalling  here we go  <end>
<start> vậy  không trì hoãn nữa bắt đầu thôi  <end> <pad>
<start>  becomes cookie fission  cookie fission is   <end> <pad>
<start>  trở thành cookie fission  cookie fission là   <end>
<start> so here aposs gdp per person in america  <end> <pad> <pad>
<start> đây chính là gdp của mỗi người ở mỹ  <end> <pad>
<start> but you know  watson is growing up fast  <end> <pad>
<start> nhưng các bạn biết watson đang phát triển nhanh chóng <end> <pad>
<start> but all of these attempts have one thing in common  <end>
<start> nhưng tất cả những nỗ lực ấy có một điểm chung <end>
<start

In [11]:
x_train, x_val, y_train, y_val = train_test_split(inp_tensor, tar_tensor, test_size=0.1)

BATCH_SIZE = 32
BUFFER_SIZE = x_train.shape[0]
N_BATCH = BUFFER_SIZE//BATCH_SIZE
hidden_unit = 1024
embedding_size = 256
print(BUFFER_SIZE)


dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.batch(BATCH_SIZE)

5414


In [12]:
tmp_x, tmp_y = next(iter(dataset))
print(tmp_x.shape)
print(tmp_y.shape)

(32, 13)
(32, 13)


In [13]:
class Encode(tf.keras.Model):
    def __init__(self, embedding_size, vocab_size, hidden_units):
        super(Encode, self).__init__()
        self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.GRU = tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform')
        self.hidden_units = hidden_units
        
    def call(self, x, hidden_state):
        try:
            x = self.Embedding(x)
        except:
            print(x, print(inp_lang.vocab_size))          
        outputs, last_state = self.GRU(x, hidden_state)
        return outputs, last_state
    
    def init_hidden_state(self, batch_size):
        return tf.zeros([batch_size, self.hidden_units])

In [14]:
encoder = Encode(embedding_size, inp_lang.vocab_size, hidden_unit)
hidden_state = encoder.init_hidden_state(BATCH_SIZE)
tmp_outputs, last_state = encoder(tmp_x, hidden_state)
print(tmp_outputs.shape)
print(last_state.shape)

(32, 13, 1024)
(32, 1024)


In [15]:
class Attention(tf.keras.Model):
    def __init__(self, hidden_units):
        super(Attention, self).__init__()
        self.W_out_encode = tf.keras.layers.Dense(hidden_unit)
        self.W_state = tf.keras.layers.Dense(hidden_unit)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, encode_outs, pre_state):
        pre_state = tf.expand_dims(pre_state, axis=1)
        pre_state = self.W_state(pre_state)
        encode_outs = self.W_out_encode(encode_outs)
        score = self.V(
            tf.nn.tanh(
                pre_state + encode_outs)
        )
        score = tf.nn.softmax(score, axis=1)
        context_vector = score*encode_outs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, score

In [16]:
attention = Attention(hidden_unit)
context_vector, attention_weight = attention(tmp_outputs, last_state)
print(context_vector.shape, attention_weight.shape)

(32, 1024) (32, 13, 1)


In [21]:
from tensorflow.python.keras.layers import recurrent
class Decode(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_units):
        super(Decode, self).__init__()
        self.hidden_units = hidden_units
        self.Embedding = tf.keras.layers.Embedding(vocab_size,embedding_size)
        self.Attention = Attention(hidden_units)
        self.GRU = tf.keras.layers.GRU(
            hidden_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        self.Fc = tf.keras.layers.Dense(vocab_size)
            
    def call(self, x, encode_outs, pre_state):
        x = tf.expand_dims(x, axis=1)
        try:
            x = self.Embedding(x)
        except:
            print(x, print(tar_lang.vocab_size))          
        context_vector, attention_weight = self.Attention(encode_outs, pre_state)
        context_vector = tf.expand_dims(context_vector, axis=1)
        gru_inp = tf.concat([x, context_vector], axis=-1)
        out_gru, state = self.GRU(gru_inp)
        out_gru = tf.reshape(out_gru, (-1, out_gru.shape[2]))
        return self.Fc(out_gru), state
    
    
decode = Decode(tar_lang.vocab_size, embedding_size, hidden_unit)
print(last_state.shape, tmp_outputs.shape, tmp_y[:, 0].shape)
decode_out, state = decode(tmp_y[:, 0], tmp_outputs, last_state)

(32, 1024) (32, 13, 1024) (32,)


In [22]:
def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [25]:
EPOCHS = 10
# optimizer = tf.train.AdamOptimizer()
optimizer = tf.optimizers.Adam()
encoder = Encode(embedding_size, vocab_size=inp_lang.vocab_size, hidden_units=hidden_unit)
decoder = Decode(vocab_size=tar_lang.vocab_size, embedding_size=embedding_size, hidden_units=hidden_unit)

In [26]:
for epoch in range(EPOCHS):
    total_loss = 0
    for batch_id, (x, y) in enumerate(dataset.take(N_BATCH)):
        loss = 0
        with tf.GradientTape() as tape:
            first_state = encoder.init_hidden_state(batch_size=BATCH_SIZE)
            encode_outs, last_state = encoder(x, first_state)
            decode_state = last_state
            decode_input = [tar_lang.word2id["<start>"]]*BATCH_SIZE
            
            for i in range(1, y.shape[1]):
                decode_out, decode_state = decoder(decode_input, encode_outs, decode_state)
                loss += loss_function(y[:, i], decode_out)
                decode_input = y[:, i]
                
            train_vars = encoder.trainable_variables + decoder.trainable_variables
            grads = tape.gradient(loss, train_vars)
            optimizer.apply_gradients(zip(grads, train_vars))
        total_loss += loss
    print(total_loss.numpy())

10623.204
9063.127
8131.009
7305.0083
6525.9995
5809.744
5131.504
4571.126
3989.8508
3586.5583


In [27]:
# 10592.284
# 9064.408
# 8132.6377
# 7290.939
# 6500.7266
# 5782.4717
# 5174.547
# 4558.6675
# 3989.8186

In [28]:
def translate(inputs):
    print(inp_lang.vector_to_sentence(inputs[0].numpy()))
    result = ''

    hidden = encoder.init_hidden_state(batch_size=1)
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    print(enc_out.shape, dec_hidden.shape)
    
    dec_input = [tar_lang.word2id['<start>']]
    for t in range(tar_lang.max_len):
        predictions, dec_hidden = decoder(dec_input, enc_out, dec_hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += tar_lang.id2word[predicted_id] + ' '
        dec_input = [predicted_id]
    return result
  
for inp, tar in dataset.take(N_BATCH):
    print(translate(inp[1:2,:]))
    break

<start> the idea of eliminating poverty is a great goal  <end> <pad>
(1, 13, 1024) (1, 1024)
ý tưởng xoá nghèo  <end>  <end>  <end>  <end>  
